<a href="https://colab.research.google.com/github/UtkarshRasal/MusicSearchAI/blob/master/MusicSearchWithDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir data

In [2]:
!mkdir faiss

In [1]:
# !pip install transformers sentence-transformers faiss-cpu pandas
!pip install faiss-cpu pandas

In [2]:
import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

In [25]:
model = SentenceTransformer('BAAI/bge-base-en')
df = pd.read_csv("/content/data/tcc_ceds_music.csv")

# drop missing data
df = df.dropna(subset=['lyrics', 'track_name', 'artist_name'])

# Fill valence with 0.5 if missing (neutral)
if 'valence' not in df.columns:
    df['valence'] = 0.5
else:
    df['valence'] = df['valence'].fillna(0.5)

# Combine fields into a searchable text
def make_text(row):
    return (
        f"{row['track_name']} by {row['artist_name']}. "
        f"Genre: {row['genre']}. "
        f"Valence: {row['valence']:.2f}. "
        f"Lyrics: {row['lyrics']}"
    )

df['combined'] = df.apply(make_text, axis=1)
df




modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/719 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

,Unnamed: 0,artist_name,track_name,release_date,genre,lyrics,len,dating,violence,world/life,...,feelings,danceability,loudness,acousticness,instrumentalness,valence,energy,topic,age,combined
0,0,mukesh,mohabbat bhi jhoothi,1950,pop,hold time feel break feel untrue convince spea...,95,0.000598,0.063746,0.000598,...,0.117175,0.357739,0.454119,0.997992,0.901822,0.339448,0.137110,sadness,1.000000,mohabbat bhi jhoothi by mukesh. Genre: pop. Va...
1,4,frankie laine,i believe,1950,pop,believe drop rain fall grow believe darkest ni...,51,0.035537,0.096777,0.443435,...,0.001284,0.331745,0.647540,0.954819,0.000002,0.325021,0.263240,world/life,1.000000,i believe by frankie laine. Genre: pop. Valenc...
2,6,johnnie ray,cry,1950,pop,sweetheart send letter goodbye secret feel bet...,24,0.002770,0.002770,0.002770,...,0.225422,0.456298,0.585288,0.840361,0.000000,0.351814,0.139112,music,1.000000,cry by johnnie ray. Genre: pop. Valence: 0.35....
3,10,pérez prado,patricia,1950,pop,kiss lips want stroll charm mambo chacha merin...,54,0.048249,0.001548,0.001548,...,0.001548,0.686992,0.744404,0.083935,0.199393,0.775350,0.743736,romantic,1.000000,patricia by pérez prado. Genre: pop. Valence: ...
4,12,giorgos papadopoulos,apopse eida oneiro,1950,pop,till darling till matter know till dream live ...,48,0.001350,0.001350,0.417772,...,0.001350,0.291671,0.646489,0.975904,0.000246,0.597073,0.394375,romantic,1.000000,apopse eida oneiro by giorgos papadopoulos. Ge...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28367,82447,mack 10,10 million ways,2019,hip hop,cause fuck leave scar tick tock clock come kno...,78,0.001350,0.001350,0.001350,...,0.001350,0.889527,0.759711,0.062549,0.000000,0.751649,0.695686,obscene,0.014286,10 million ways by mack 10. Genre: hip hop. Va...
28368,82448,m.o.p.,ante up (robbin hoodz theory),2019,hip hop,minks things chain ring braclets yap fame come...,67,0.001284,0.001284,0.035338,...,0.001284,0.662082,0.789580,0.004607,0.000002,0.922712,0.797791,obscene,0.014286,ante up (robbin hoodz theory) by m.o.p.. Genre...
28369,82449,nine,whutcha want?,2019,hip hop,get ban get ban stick crack relax plan attack ...,77,0.001504,0.154302,0.168988,...,0.001504,0.663165,0.726970,0.104417,0.000001,0.838211,0.767761,obscene,0.014286,whutcha want? by nine. Genre: hip hop. Valence...
28370,82450,will smith,switch,2019,hip hop,check check yeah yeah hear thing call switch g...,67,0.001196,0.001196,0.001196,...,0.001196,0.883028,0.786888,0.007027,0.000503,0.508450,0.885882,obscene,0.014286,switch by will smith. Genre: hip hop. Valence:...


In [27]:
# Create embeddings
embeddings = model.encode(df['combined'].tolist(), normalize_embeddings=True)
embeddings = np.array(embeddings).astype('float32')

# Save embeddings with FAISS
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

KeyboardInterrupt: 

In [8]:
import os

# Save index and metadata
# create a folder "faiss" if some faiss error occurs
os.makedirs("faiss_index", exist_ok=True)
faiss.write_index(index, "faiss_index/mendeley_index.faiss")

# Save metadata
df[['track_name', 'artist_name', 'genre', 'valence', 'lyrics']].to_csv("faiss_index/metadata.csv", index=False)

In [17]:
def load_resources():
    index = faiss.read_index("faiss_index/mendeley_index.faiss")
    df = pd.read_csv("faiss_index/metadata.csv")
    return index, df

def search(query, top_k=5):
    query_vec = model.encode([query], normalize_embeddings=True).astype("float32")
    D, I = index.search(query_vec, top_k)
    # return df.iloc[I[0]]

    print(f"\n🔍 Top {top_k} Results for: '{query}'\n")
    for rank, idx in enumerate(I[0]):
        row = df.iloc[idx]
        print(f"{rank+1}. {row['track_name']} – {row['artist_name']}")
        print(f"    Lyrics: {row['lyrics'][:100]}...\n")

# Load
index, df = load_resources()

In [24]:
# Try example searches
print(search("sad romantic piano song"))
print('=========================================================================')
print(search("happy energetic dance song"))
print('=========================================================================')
print(search("lonely heartbreak lyrics"))
print('=========================================================================')
print(search("song by frankie laine"))
print('=========================================================================')
print(search("most emotional song"))


🔍 Top 5 Results for: 'sad romantic piano song'

1. piano in the dark – brenda russell
    Lyrics: watch time think funny things say feel like dead lead turn room know gonna longer feel stronger walk...

2. moonlight and music – ruby and the romantics
    Lyrics: moonlight music starlight cry trouble go go leave memory good moonlight moonlight music write letter...

3. sad songs and waltzes – willie nelson
    Lyrics: write song true song real tear need fear cause hear cause songs waltz aren sell year tell cheat like...

4. sad, sad music – dwight yoakam
    Lyrics: music music silence leave miss couple days forget go wrong fell apart know tell leavin cross mind he...

5. the way it used to be – engelbert humperdinck
    Lyrics: lonely table bright crowd room music begin drink memories gloom music bittersweet refrain play song ...

None

🔍 Top 5 Results for: 'happy energetic dance song'

1. you happy puppet – 10,000 maniacs
    Lyrics: teach puppet dance string learn come slavish funne